In [1]:
import os
from concurrent.futures import ProcessPoolExecutor
# set up Whitebox Environment, get available functions
from whitebox_workflows import WbEnvironment
from scripts.constants import create_scenes_df, dem_fill
from scripts.constants import flow_direction, flow_accumulation
import shutil
import multiprocessing
import rasterio
from rasterio.plot import show
from osgeo import gdal

In [2]:
logical_cores = multiprocessing.cpu_count()
# determine number of threads to use for multiprocessing
num_workers = int(logical_cores * 0.8)  # rounds down in case not a whole number
print(f'Number of threads to use: {num_workers}')

wbe = WbEnvironment()

scenes_df = create_scenes_df()

Number of threads to use: 8


In [3]:
data_folder = 'data'
input_dem = os.path.join(data_folder, scenes_df.at[0, 'scene_folder'],scenes_df.at[0, 'input_dem'])
print(input_dem)

temp_dir = 'data/temp_dir'

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print(f'{temp_dir} already exists')

data/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_dem.tif
data/temp_dir already exists


In [ ]:
# # # copy the dem to the directory
# dem_copy = f'{temp_dir}/dem_copy.tif'
# print(input_dem)
# shutil.copyfile(input_dem, dem_copy)



If you inspect the DEM, which is the same in both folders, it is much larger in area converage than the scenes.  Because of this extra area, we would be processing data unnecessarily, so I'm going to geometrically clip DEM file to the scene grid.

We will use the Geospatial Data Abstraction Library ([GDAL](https://gdal.org/en/stable/index.html), pronounced *\`gee doll\`*).

The documentation is difficult to navigate.   Checkout this [cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/).  

In [5]:
# grab shapefile to clip the dem
cutline_shapefile = os.path.join(data_folder, scenes_df.at[0, 'scene_folder'], scenes_df.at[0, 'shapefile'])

# get the shapefile's layer name
cutline_layer = os.path.splitext(os.path.basename(cutline_shapefile))[0]
dem_clipped = os.path.join(temp_dir, 'dem_clipped.tif')
print(f'Shapefile: {cutline_shapefile}\nShape Layer: {cutline_layer}\nOutput file: {dem_clipped}')

Shapefile: data/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A/S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_shape.shp
Shape Layer: S1A_IW_20250205T233956_DVP_RTC10_G_gdufem_246A_shape
Output file: data/temp_dir/dem_clipped.tif


In [12]:
# run help for the gdal.WarpOptions()
help(gdal.WarpOptions)

Help on function WarpOptions in module osgeo.gdal:

WarpOptions(
    options=None,
    format=None,
    srcBands=None,
    dstBands=None,
    outputBounds=None,
    outputBoundsSRS=None,
    xRes=None,
    yRes=None,
    targetAlignedPixels=False,
    width=0,
    height=0,
    srcSRS=None,
    dstSRS=None,
    coordinateOperation=None,
    srcAlpha=None,
    dstAlpha=False,
    warpOptions=None,
    errorThreshold=None,
    warpMemoryLimit=None,
    creationOptions=None,
    outputType=0,
    workingType=0,
    resampleAlg=None,
    srcNodata=None,
    dstNodata=None,
    multithread=False,
    tps=False,
    rpc=False,
    geoloc=False,
    polynomialOrder=None,
    transformerOptions=None,
    cutlineDSName=None,
    cutlineWKT=None,
    cutlineSRS=None,
    cutlineLayer=None,
    cutlineWhere=None,
    cutlineSQL=None,
    cutlineBlend=None,
    cropToCutline=False,
    copyMetadata=True,
    metadataConflictValue=None,
    setColorInterpretation=False,
    overviewLevel='AUTO',
  

In [ ]:
# use gdalwarp to clip the data
gdal.Warp(
    dem_clipped, input_dem,                     # output file, input file
    format="COG",                               # output format - Cloud-Optimized Geotiff
    cutlineDSName=cutline_shapefile,            # Geometry to use for clipping
    cutlineLayer=cutline_layer,                 # name of geometry data
    dstNodata=-9999,                            # assign obscure value to nodata, this is important for later processing
    multithread=num_workers                     # using multiple cpus to perform task
)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x11461edc0> >

TODO, fill missing data.


Call `dem_fill` function from *constants.py* script.  This will
 - find pits
 - fill depressions
 - find flats on flow
 - fix flats

Process can take 10 minutes or so.

In [ ]:
# Fill pits
# read the raster
dem = wbe.read_raster(dem_copy)
pit_filled_dem = wbe.fill_pits(dem)
output_file = os.path.join(temp_dir, 'pit_filled_dem.tif')
pit_filled_dem = wbe.write_raster(pit_filled_dem, output_file)

In [ ]:
pit_filled_dem = f'{temp_dir}/pit_filled_dem.tif'

In [ ]:
# I'm setting this up to run on multiple CPUs, even though here we are only using one.  
# The idea is that in the future, I may do this on several files
if not os.path.exists(f'{temp_dir}/filled_dem.tif'):
    print("file does not exist")
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(dem_fill, pit_filled_dem, temp_dir)
        input_dem_raster = future.result()
else:
    print('Filled Dem Tiff file already exists')  

#### Generate Flow direction
Create a flow direction model from the Filled Dem we just created.
Runs in seconds

In [ ]:
d8_pointer = f'{temp_dir}/d8_pointer.tif'

if not os.path.exists('data/temp_dir/dem_flow.tif'):
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(flow_direction, d8_pointer, temp_dir)
else:
    print(f'flow_direction already exists.')

#### Generate Flow Accumulation

In [ ]:
flow_accum = os.path.join(temp_dir, 'flow_accum.tif')
flow_dem = os.path.join(temp_dir, 'flow_dem.tif')

if not os.path.exists(flow_accum):
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        future = executor.submit(flow_accumulation, flow_direction, temp_dir)